In [1]:
!pip install fasttext
!pip install scikit-multilearn

In [2]:
import pandas as pd
import numpy as np
import fasttext.util
from collections import defaultdict
import ast


In [3]:
fasttext.util.download_model('hi', if_exists='ignore')  # Hindi
ft = fasttext.load_model('cc.hi.300.bin')

In [4]:
train_file = 'train.csv'
val_file = 'val.csv'
test_file = 'test.csv'

In [5]:
train_data = pd.read_csv(train_file, header=0, index_col=0)
val_data = pd.read_csv(val_file, header=0, index_col=0)
non_hostile_val_data = pd.read_csv('y_pred_val_non_hostile.csv', header=0, index_col=0)
test_data = pd.read_csv(test_file, header=0, index_col=0)
train_val_data = train_data.append(val_data, ignore_index=True)

In [6]:
non_hostile_test_data = pd.read_csv('y_pred_test_non_hostile.csv', header=0, index_col=0)

In [7]:
train_data.drop(train_data[train_data['Labels Set']=='non-hostile'].index, inplace = True)
val_data.drop(non_hostile_val_data[non_hostile_val_data['non-hostile']==1].index, inplace=True)
train_val_data.drop(train_val_data[train_val_data['Labels Set']=='non-hostile'].index, inplace = True)

In [8]:
test_data.drop(non_hostile_test_data[non_hostile_test_data['non-hostile']==1].index, inplace=True)

In [9]:
print(train_data.shape)
train_data.head()

(2678, 13)


,Post,Labels Set,emails,urls,mentions,hashtags,emojis,emoticons,reserved_words,Filtered_Post,Filtered_Post_Stopword_Removed,Unnamed: 12,Unnamed: 13
Unique ID,,,,,,,,,,,,,
1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive",[],[],[],[],"['🙏', '🙏']",[],[],मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,मेरे देश हिन्दु निराले है। पक्के राम भक्त बाबर...,NaN,NaN
4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive",[],[],['@prabhav218'],[],[],[],[],साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,साले जेएनयू छाप कमिने लोग हिन्दुओं संविधान सबक...,NaN,NaN
6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अ...,fake,[],[],[],[],[],[],[],चीन ने UN में तर्क दिया की भारत का विपक्ष ही अ...,चीन UN तर्क भारत विपक्ष अजर‌ मसुद आतंकी मानता ...,NaN,NaN
11,RT @_Pb_swain_: मोदीजी और जब सारा देश सेना के ...,hate,[],[],['@_Pb_swain_:'],[],['🤔'],[],['RT'],"मोदीजी और जब सारा देश सेना के साथ खडी है,\n\...","मोदीजी देश सेना खडी है, सयाने विदेश पडे है? बो...",NaN,NaN
12,RT @ShilpiSinghINC: 48000 घरों को तोड़ने का आद...,offensive,[],[],['@ShilpiSinghINC:'],[],[],[],['RT'],"48000 घरों को तोड़ने का आदेश आया है, किसी को...","48000 घरों तोड़ने आदेश आया है, फर्क़ पड़ता! अम...",NaN,NaN


In [10]:
print(val_data.shape)
val_data.head()

(380, 11)


,Post,Labels Set,emails,urls,mentions,hashtags,emojis,emoticons,reserved_words,Filtered_Post,Filtered_Post_Stopword_Removed
Unique ID,,,,,,,,,,,
2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation,[],[],[],[],[],[],[],भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,भारतीय जनता पार्टी rss इतने गिरे हूं मेरी जासू...
8,अद्भुत - जो वामपंथी कहते है कि महाभारत का युद्...,fake,[],[],[],[],[],[],[],अद्भुत - जो वामपंथी कहते है कि महाभारत का युद्...,"अद्भुत - वामपंथी महाभारत युद्ध काल्पनिक है, सन..."
13,भाई जाके हिन्दू भाइयों की मदद कर जिनकी नौकरी च...,offensive,[],[],[],[],[],[],[],भाई जाके हिन्दू भाइयों की मदद कर जिनकी नौकरी च...,भाई जाके हिन्दू भाइयों मदद जिनकी नौकरी चली आत्...
14,यह दुर्घटना कन्नूर अंतर्राष्ट्रीय हवाई अड्डे म...,fake,[],[],[],[],[],[],[],यह दुर्घटना कन्नूर अंतर्राष्ट्रीय हवाई अड्डे म...,"दुर्घटना कन्नूर अंतर्राष्ट्रीय हवाई अड्डे हुई,..."
15,सत्य कभी कमजोर नही होने देता। और “ज्ञान” कभी द...,non-hostile,[],[],[],[],[],[],[],सत्य कभी कमजोर नही होने देता। और “ज्ञान” कभी द...,सत्य कमजोर नही देता। “ज्ञान” दुःखी भयभीत नही द...


In [11]:
print(test_data.shape)
test_data.head()

(759, 10)


,Post,emails,urls,mentions,hashtags,emojis,emoticons,reserved_words,Filtered_Post,Filtered_Post_Stopword_Removed
Unique ID,,,,,,,,,,
1,कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...,[],[],[],[],[],[],[],कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...,कीस रोजगार कहना रोजगार मिलता 20 करोड रोजगार 6 ...
3,"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...",[],[],[],[],"['🙏', '😂', '👍']",[],[],"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...","कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊंची बिल..."
4,अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...,[],[],[],[],[],[],[],अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...,अंडरवर्ल्ड डॉन छोटा राजन भाई बीजेपी टिकट मिला है।
5,RT @_Pb_swain_: इन पंचर छापों को कोन समझाए कि ...,[],[],['@_Pb_swain_:'],[],"['👇', '😂', '😂', '😂', '😂']",[],['RT'],इन पंचर छापों को कोन समझाए कि उनके रोजगार मे...,पंचर छापों समझाए रोजगार कमी कारण मोदी ट्यूब लै...
8,@BasudebaTripat4: @Rajanspsingh1 अच्छा किया सा...,[],[],"['@BasudebaTripat4:', '@Rajanspsingh1']",[],[],[],[],"अच्छा किया साले का सर फोड़ दिया,, गर्दन तोड़...","अच्छा साले सर फोड़ दिया,, गर्दन तोड़ देते अच्छ..."


In [12]:
print(train_val_data.shape)
train_val_data.head()

(3054, 13)


,Post,Labels Set,emails,urls,mentions,hashtags,emojis,emoticons,reserved_words,Filtered_Post,Filtered_Post_Stopword_Removed,Unnamed: 12,Unnamed: 13
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive",[],[],[],[],"['🙏', '🙏']",[],[],मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,मेरे देश हिन्दु निराले है। पक्के राम भक्त बाबर...,NaN,NaN
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive",[],[],['@prabhav218'],[],[],[],[],साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,साले जेएनयू छाप कमिने लोग हिन्दुओं संविधान सबक...,NaN,NaN
5,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अ...,fake,[],[],[],[],[],[],[],चीन ने UN में तर्क दिया की भारत का विपक्ष ही अ...,चीन UN तर्क भारत विपक्ष अजर‌ मसुद आतंकी मानता ...,NaN,NaN
10,RT @_Pb_swain_: मोदीजी और जब सारा देश सेना के ...,hate,[],[],['@_Pb_swain_:'],[],['🤔'],[],['RT'],"मोदीजी और जब सारा देश सेना के साथ खडी है,\n\...","मोदीजी देश सेना खडी है, सयाने विदेश पडे है? बो...",NaN,NaN
11,RT @ShilpiSinghINC: 48000 घरों को तोड़ने का आद...,offensive,[],[],['@ShilpiSinghINC:'],[],[],[],['RT'],"48000 घरों को तोड़ने का आदेश आया है, किसी को...","48000 घरों तोड़ने आदेश आया है, फर्क़ पड़ता! अम...",NaN,NaN


**Transforming the data**

In [13]:
labels_set = {'defamation',
 'fake',
 'hate',
 'non-hostile',
 'offensive'}

labels_mapping = {'defamation':0,
 'fake':1,
 'hate':2,
 'non-hostile':3,
 'offensive':4}

In [14]:
train_y = np.empty((0, 5))
for index, row in train_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  train_y = np.vstack((train_y, y))


val_y = np.empty((0, 5))
for index, row in val_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  val_y = np.vstack((val_y, y))

train_val_y = np.empty((0, 5))
for index, row in train_val_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  train_val_y = np.vstack((train_val_y, y))


In [15]:
print(train_y.shape)
print(val_y.shape)
print(train_val_y.shape)

(2678, 5)
(380, 5)
(3054, 5)


In [16]:
mentions_dict = defaultdict(int)

for index, row in train_data.iterrows():
  if 'defamation' in row['Labels Set']:
    mentions = ast.literal_eval(row['mentions']) 
    for m in mentions:
      mentions_dict[m]+=1


In [17]:
filtered_mentions_list = []

for key, value in mentions_dict.items():
    if value > 2:
      filtered_mentions_list.append(key)

In [18]:
offensive_words_list = ['कुत्ते','कुत्ते','कुतिया', 'कुते', 'कुत्ती', 'कुत्तों', 'कमीने', 'कमीना', 'कमीनी', 'साले', 'साला', 'साली', 'हरामी', 'हरामखोर', 'बहनचोद', 'बहिनचोद', 'भेंचो', 'मादरचोद', 'माचोद', 'चूतिये', 'चूतिया', 'चुत', 'चूत', 'लवड़ा', 'लंड', 'लन्ड', 'लँड', 'लण्ड', 'लौड़ा', 'लोडे', 'भड़वो', 'भड़ुआ', 'भड़ऊ', 'भड़वे', 'भड्वे', 'रन्डी', 'रांड', 'भोसड़ी', 'गांड', 'बेटीचोद', 'बेटिचोद', 'फट्टू', 'फुददू', 'फुद्दु', 'सुअर', 'झांट', 'ना󰀂ाय󰀂', 'टट्टी', 'ह󰀂󰀂󰀂डे', 'हुरो', 'गद्दार', 'सूअरों', 'हरामि', 'गाली', 'गॉड', 'रडुआ', 'लुच्चे', 'भड़वागिरी', 'बेवकूफ', 'कुत्तो', 'मूर्खों', 'बकवास', 'तब्लीघि', 'चोरी']

In [19]:
mention_len = len(filtered_mentions_list)

In [20]:
mention_len

20

In [21]:
hashtag_dict = defaultdict(int)

for index, row in train_data.iterrows():
  if 'defamation' in row['Labels Set']:
    mentions = ast.literal_eval(row['hashtags']) 
    for m in mentions:
      hashtag_dict[m]+=1


In [22]:
filtered_hashtag_list = []

for key, value in hashtag_dict.items():
    if value > 2:
      filtered_hashtag_list.append(key)

In [23]:
hash_len = len(filtered_hashtag_list)

In [24]:
hash_len

5

In [25]:
emoji_dict = defaultdict(int)

for index, row in train_data.iterrows():
  if 'defamation' in row['Labels Set']:
    emojis = ast.literal_eval(row['emojis']) 
    for e in emojis:
      emoji_dict[e]+=1


In [26]:
filtered_emoji_list = []

for key, value in emoji_dict.items():
  if value > 5:
    filtered_emoji_list.append(key)

In [27]:
emoji_len = len(filtered_emoji_list)

In [28]:
emoji_len

16

In [29]:
def generate_matrix(data):
  X = np.empty((0, 300 + 1 + hash_len + mention_len+emoji_len +len(offensive_words_list)))

  for index, row in data.iterrows():
    x = np.zeros((1, 301))
    x[0, 0] = 1  # Bias Unit

    # Word Embeddings
    for word in row['Filtered_Post_Stopword_Removed'].split(' '):
      x[0, 1:301]+= ft[word]

    
    # Offensive Words
    offensive_word_count_x = np.zeros((1, len(offensive_words_list)))
    for word in row['Filtered_Post_Stopword_Removed'].split(' '):
      if word in offensive_words_list:
        offensive_word_count_x[0,offensive_words_list.index(word)]= 1

    hash_x = np.zeros((1, hash_len))
    # Hashtags
    for h in ast.literal_eval(row['hashtags']):
      if h in filtered_hashtag_list:
        hash_x[0, filtered_hashtag_list.index(h)] = 1

    
    mention_x = np.zeros((1, mention_len))
    # Mentions
    for h in ast.literal_eval(row['mentions']):
      if m in filtered_mentions_list:
        mention_x[0, filtered_mentions_list.index(m)] = 1

    emoji_x = np.zeros((1, emoji_len))
    # Emojis
    for h in ast.literal_eval(row['emojis']):
      if h in filtered_emoji_list:
        emoji_x[0, filtered_emoji_list.index(h)] = 1

    x = np.hstack((x, hash_x, mention_x, emoji_x, offensive_word_count_x))
    X = np.vstack((X, x))

  return X


In [30]:
train_X = generate_matrix(train_data)
val_X = generate_matrix(val_data)

In [31]:
val_X.shape

(380, 406)

In [32]:
# Label Mapping
labels_mapping = {'defamation':0,
 'fake':1,
 'hate':2,
 'non-hostile':3,
 'offensive':4}


In [33]:
train_y = np.empty((0, 5))
for index, row in train_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  train_y = np.vstack((train_y, y))


val_y = np.empty((0, 5))
for index, row in val_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  val_y = np.vstack((val_y, y))




In [34]:
y_train_defamation = train_y[:,0].astype(int)
y_val_defamation = val_y[:,0].astype(int)

In [35]:
def evaluation(y_true, y_pred):
  print("Fine Grained Accuracy = {}".format(accuracy_score(y_true, y_pred)))
  print("\n\nFine Grained Metrics\n")
  print(classification_report(y_true, y_pred))
  

**Model**

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
!pip install xgboost
from xgboost import XGBClassifier

In [37]:
classifier =SVC(class_weight='balanced')
classifier.fit(train_X, y_train_defamation)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [38]:
predictions = classifier.predict(val_X)

In [39]:
evaluation(y_val_defamation.reshape(-1,1), predictions)

Fine Grained Accuracy = 0.7263157894736842


Fine Grained Metrics

              precision    recall  f1-score   support

           0       0.91      0.74      0.81       307
           1       0.38      0.68      0.49        73

    accuracy                           0.73       380
   macro avg       0.64      0.71      0.65       380
weighted avg       0.81      0.73      0.75       380



In [40]:
result_df = pd.DataFrame(predictions, index = val_data.index, columns=['defamation'])
result_df.index.name = 'Unique ID'
result_df.to_csv('y_pred_val_defamation.csv')

**Evaluation on Test Data**

In [41]:
mentions_dict = defaultdict(int)

for index, row in train_val_data.iterrows():
  if 'defamation' in row['Labels Set']:
    mentions = ast.literal_eval(row['mentions']) 
    for m in mentions:
      mentions_dict[m]+=1


In [42]:
filtered_mentions_list = []

for key, value in mentions_dict.items():
    if value > 1:
      filtered_mentions_list.append(key)

In [43]:
mention_len = len(filtered_mentions_list)

In [44]:
mention_len

50

In [45]:
hashtag_dict = defaultdict(int)

for index, row in train_val_data.iterrows():
  if 'defamation' in row['Labels Set']:
    mentions = ast.literal_eval(row['hashtags']) 
    for m in mentions:
      hashtag_dict[m]+=1


In [46]:
filtered_hashtag_list = []

for key, value in hashtag_dict.items():
    if value > 2:
      filtered_hashtag_list.append(key)

In [47]:
hash_len = len(filtered_hashtag_list)

In [48]:
hash_len

6

In [49]:
emoji_dict = defaultdict(int)

for index, row in train_val_data.iterrows():
  if 'defamation' in row['Labels Set']:
    emojis = ast.literal_eval(row['emojis']) 
    for e in emojis:
      emoji_dict[e]+=1


In [50]:
filtered_emoji_list = []

for key, value in emoji_dict.items():
  if value > 3:
    filtered_emoji_list.append(key)

In [51]:
emoji_len = len(filtered_emoji_list)

In [52]:
emoji_len

28

In [53]:
def generate_matrix(data):
  X = np.empty((0, 300 + 1 + hash_len + mention_len+emoji_len))

  for index, row in data.iterrows():
    x = np.zeros((1, 301))
    x[0, 0] = 1  # Bias Unit

    # Word Embeddings
    for word in row['Filtered_Post'].split(' '):
      x[0, 1:301]+= ft[word]

    hash_x = np.zeros((1, hash_len))
    # Hashtags
    for h in ast.literal_eval(row['hashtags']):
      if h in filtered_hashtag_list:
        hash_x[0, filtered_hashtag_list.index(h)] = 1

    
    mention_x = np.zeros((1, mention_len))
    # Mentions
    for h in ast.literal_eval(row['mentions']):
      if m in filtered_mentions_list:
        mention_x[0, filtered_mentions_list.index(m)] = 1

    emoji_x = np.zeros((1, emoji_len))
    # Emojis
    for h in ast.literal_eval(row['emojis']):
      if h in filtered_emoji_list:
        emoji_x[0, filtered_emoji_list.index(h)] = 1

    x = np.hstack((x, hash_x, mention_x, emoji_x))
    X = np.vstack((X, x))

  return X


In [54]:
train_val_X = generate_matrix(train_val_data)
test_X = generate_matrix(test_data)

In [55]:
test_X.shape

(759, 385)

In [56]:
# Label Mapping
labels_mapping = {'defamation':0,
 'fake':1,
 'hate':2,
 'non-hostile':3,
 'offensive':4}


In [57]:
y_train_val_defamation = train_val_y[:,0].astype(int)

In [63]:
def evaluation(y_true, y_pred):
  print("Fine Grained Accuracy = {}".format(accuracy_score(y_true, y_pred)))
  print("\n\nFine Grained Metrics\n")
  print(classification_report(y_true, y_pred))
  

**Model**

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
!pip install xgboost
from xgboost import XGBClassifier

In [60]:
classifier =SVC(class_weight='balanced')
classifier.fit(train_val_X, y_train_val_defamation)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [61]:
predictions = classifier.predict(test_X)

In [62]:
result_df = pd.DataFrame(predictions, index = test_data.index, columns=['defamation'])
result_df.index.name = 'Unique ID'
result_df.to_csv('y_pred_test_defamation.csv')

In [65]:
filename = 'defamation_test.sav'
import pickle


In [66]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [68]:
path = F"/content/gdrive/My Drive/{filename}" 
pickle.dump(classifier, open(path, 'wb'))